# Importing Masterliste Werke+Aufnahmen

The sheet "machine readable" contains the fixed data including manually added folder and file names for csv data files (tappings)

## Preparation
Download sheet "machine readable" as csv.


# Adding data from notebooks

## Audio conversion

In [3]:
from pydub import AudioSegment

data_dir = "/home/martin/Workspace/musicData/data/recordings/BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2]"
file_name = data_dir + "/" + "03_BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2].flac"

song = AudioSegment.from_file(file_name, "flac")
song.export(file_name + ".wav", format="wav")



<_io.BufferedRandom name='/home/martin/Workspace/musicData/data/recordings/BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2]/03_BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2].flac.wav'>

## OMA API

In [40]:
import requests
import json

# url = "http://omatest.eu-central-1.elasticbeanstalk.com:8080/"
url = "http://localhost:8080/"

_headers = {
    'User-Agent': 'Bootstrap.py',
    'Authorization': ""
}
_endpoint = {
    "uri": ""
}

_config = {
    "API_PATH": "api/"
}

def login(endpoint, username, password):
    """
    Authenticates to marketme.marketme.universalimporter and sets the Authorization
    header field.
    :param username: a valid username for marketme.marketme.universalimporter
    :param password: the password
    :return: http response
    """
    _endpoint["uri"] = endpoint
    data = {
        'username': username,
        "password": password
    }
    url = _endpoint["uri"] + _config["API_PATH"] + "login"
    response = requests.post(url, json=data, verify=False)
    content = json.loads(response.content.decode())
    _headers['Authorization'] = content['token_type'] + " " + content['access_token']
    return response

def composer_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "composer"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content

def composition_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "composition"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content
    
def interpretation_add(params):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "interpretation"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content

def interpretation_add_abstract_music_part(params):
    params["method"] = "addAbstractMusicPart"
    url = _endpoint["uri"] + _config["API_PATH"] + "interpretation"
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    return content

def recording_add(params, files):
    params["method"] = "add"
    url = _endpoint["uri"] + _config["API_PATH"] + "recording"
    # response = requests.post(url, headers=_headers, data = params, files = files, verify=False)
    # upload without a file first. if recording exists, no upload necessary
    response = requests.post(url, headers=_headers, data = params, verify=False)
    content = json.loads(response.content.decode())
    
    if "error" in content:
        if (content["error"] == "please provide a file"):
            print("providing a file")
            response = requests.post(url, headers=_headers, data = params, files = files, verify=False)
            # response = requests.post(url, headers=_headers, data = params, verify=False)
            content = json.loads(response.content.decode())
        
    
    return content


login(url, "oma", "=PSe?sZ-ymp6mE>2")



<Response [200]>

## Using the OMA API to upload the data

In [41]:
import pandas as pd
import math
import os
from pydub import AudioSegment

data_list = "data/Masterliste Werke+Aufnahmen - machine readable.csv"
data_dir = "/home/martin/Workspace/musicData/data/"
df = pd.read_csv(data_list)

df.head()


def parse_t_csv(filename):
    tapping_f = open(filename, "r")
    # print(tapping_f.read())
    return {}

    

filtered_data = df[df['tapping_files'].notnull()]
    
for index, row in filtered_data.iterrows():
    
    tapping_file_list = row['tapping_files'].split(",")
    
    # parse row
    composer = row["Komponist"]
    work = row["Werkname"]
    part_number = row["Satz_Nr"]
    conductor = row["Dirigent"]
    orchestra = row["Orchester"]
    year = row["Aufnahmejahr"]
    
    # print(composer, work, part_number, conductor, orchestra, year)
    
    c = {"name": composer}
    c = composer_add(c)
    print(c)

    composition = {"title": work, "composer": c["id"]}
    composition = composition_add(composition)
    print(composition)
    
    # interpretation
    interpretation = {"title":  conductor + ", " + orchestra + ", " + str(year)}
    interpretation = interpretation_add(interpretation)
    print(interpretation)
    
    
    # abstract music (work) part of composition
    
    if math.isnan(row["Takt von"]):
        row["Takt von"] = 0.0
    
    if math.isnan(row["Takt bis"]):
        row["Takt bis"] = 0.0
    
    # offset and lengths in OMA instead of start and end. It's equivalent   
    row["Takt bis"] = row["Takt bis"] - row["Takt von"] 

    
    abstract_music_part = {
        "title": "Movement " + str(part_number),
        "movement": part_number,
        "composition": composition["id"],
        "interpretation": interpretation["id"],
        "bar_number_offset": row["Takt von"],
        "number_of_bars": row["Takt bis"]
    }
    abstract_music_part = interpretation_add_abstract_music_part(abstract_music_part)
    print(abstract_music_part)
    
    
    # recording and audio files
    audio_path = data_dir + "recordings/" +row["recording_folder"]
    audio_file_list = os.listdir(audio_path)
    legit_audio_files = []
    for a in audio_file_list:
        if a.endswith(".flac"): # or .wav or whatever
            legit_audio_files.append(a)
    
    if (len(legit_audio_files) != 1):
        print ("only single audio file supported")
        sys.exit(1)
    
    file_name = legit_audio_files[0]
    audio_file = audio_path + file_name
    wav_file = audio_file + ".wav"
    mp3_file = audio_file + ".64k.mp3"
    
    song = AudioSegment.from_file(audio_file, "flac")
    # song.export(wav_file, format="wav")
    
    
    if os.path.isfile(mp3_file) == False:
        print("converting")
        song.export(mp3_file, format="mp3", bitrate="64k")
        
    
    recording = {
        "title": "Recording of " + str(row["Aufnahmejahr"]),
        "interpretation": interpretation["id"],
        "abstract_music_part": abstract_music_part["id"]       
    }
    files = {
        # "file": (wav_file, open(wav_file, 'rb'), 'audio/wav', {'Expires': '0'})
        "file": (mp3_file, open(mp3_file, 'rb'), 'audio/mpeg', {'Expires': '0'})
    
    }
    
    recording = recording_add(recording, files)
    print(recording)
    
    
    print ("")
    
    # tappings. multiple files possible
    for i in range(0, len(tapping_file_list)):
        # print("\t", tapping_file_list[i])
        # import that tapping file to recording
        cwd = data_dir + row["tapping_folder"]
        tapping_data = parse_t_csv(cwd + "/" + tapping_file_list[i].rstrip())
        # upload tapping data
        
        



{'tenantId': 3, 'name': 'Anton Bruckner', 'id': 5, 'compositions': [{'id': 6}]}
{'tenantId': 3, 'title': 'Symphonie No. 9', 'composer': {'id': 5}, 'id': 6}
{'recordings': [{'id': 9}], 'title': 'Eugen Jochum, Berliner Philharmoniker, 1966', 'tenantId': 3, 'id': 7, 'abstractMusicParts': [{'id': 8}]}
{'id': 8, 'tenantId': 3, 'interpretation': {'id': 7}, 'abstractMusic': {'id': 6}, 'barNumberOffset': 150.0, 'title': 'Movement 3', 'pdfLocation': None, 'interpretationOrder': 3, 'numberOfBars': 93.0}
{'id': 9, 'digitalAudio': [{'id': 10}], 'tenantId': 3, 'recordingData': {}, 'abstractMusicPart': {'id': 8}, 'title': 'Recording of 1966', 'interpretation': {'id': 7}}

{'tenantId': 3, 'name': 'Anton Bruckner', 'id': 5, 'compositions': [{'id': 6}]}
{'tenantId': 3, 'title': 'Symphonie No. 9', 'composer': {'id': 5}, 'id': 6}
{'recordings': [{'id': 13}], 'title': 'Herbert von Karajan, Berliner Philharmoniker, 1975', 'tenantId': 3, 'id': 11, 'abstractMusicParts': [{'id': 12}]}
{'id': 12, 'tenantId': 3